# Gas Storage Valuation 

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

BASE_DIR = Path('..').resolve()
DATA_RAW = BASE_DIR / 'data' / 'raw'
RESULTS = BASE_DIR / 'results'
RESULTS.mkdir(parents=True, exist_ok=True)
print('BASE_DIR:', BASE_DIR)

BASE_DIR: C:\Users\dappy\Downloads\Public-portfolio\Energy\gas_storage_valuation


In [2]:
import sys

# notebook add-on, to find repo root by walking up until we see src
p = Path().resolve()
while p != p.parent and not (p / "src").exists():
    p = p.parent

sys.path.insert(0, str(p))
print("Added to path:", p)


Added to path: C:\Users\dappy\Downloads\Public-portfolio\Energy\gas_storage_valuation


In [3]:
import pandas as pd
import numpy as np


spot = pd.read_csv(DATA_RAW / "spot_history_ttf_like.csv", parse_dates=["date"]).sort_values("date")


fwd = pd.read_csv(DATA_RAW / "forward_curve_monthly.csv").sort_values("delivery_month")
fwd["delivery_month"] = fwd["delivery_month"].astype(str).str.strip()


dt_ym = pd.to_datetime(fwd["delivery_month"], format="%Y-%m", errors="coerce")
dt_dmy = pd.to_datetime(fwd["delivery_month"], format="%d/%m/%Y", errors="coerce")


fwd["delivery_month"] = np.where(dt_ym.notna(), dt_ym, dt_dmy)
fwd["delivery_month"] = pd.to_datetime(fwd["delivery_month"], errors="raise")

# normalise to month-start timestamp (YYYY-MM-01)
fwd["delivery_month"] = fwd["delivery_month"].dt.to_period("M").dt.to_timestamp()


fwd = (
    fwd.sort_values("delivery_month")
       .groupby("delivery_month", as_index=False)["fwd_eur_mwh"]
       .last()
)

# build mapping month_start -> forward price
fwd_map = pd.Series(fwd["fwd_eur_mwh"].values, index=fwd["delivery_month"])


In [4]:
start = max(spot["date"].iloc[-1] + pd.Timedelta(days=1), fwd["delivery_month"].min())
horizon_days = 365
dates = pd.date_range(start, periods=horizon_days, freq="D")

month_start = dates.to_period("M").to_timestamp()
price_series = pd.Series(month_start, index=dates).map(fwd_map).astype(float)

# if curve shorter than horizon, carry last observed forward price forward
price_series = price_series.ffill()

# safety check
if price_series.isna().any():
    missing = pd.Series(month_start[price_series.isna()]).unique()
    raise ValueError(f"Still missing forward prices for months: {missing[:10]}")


## Storage optimisation via Dynamic Programming
Solve for the optimal inject/withdraw/hold decision each day, given constraints.

In [5]:
from src.storage_dp import StorageParams, optimal_policy_perfect_foresight

# Storage/technical constraints
params = StorageParams(
    capacity=100.0,
    init_inventory=50.0,
    inj_rate=2.0,
    wdr_rate=2.0,
    inj_fee=0.02,
    wdr_fee=0.02,
    loss_frac=0.0,
    discount_rate_annual=0.0
)

# optimal_policy_perfect_foresight returns 
value, policy = optimal_policy_perfect_foresight(price_series, params, grid_size=200)

value


1441.9946532663237

In [6]:
print("Intrinsic value (EUR):", value)
display(policy.head(10))
display(policy.tail(10))
print("Number of decision days:", len(policy))


Intrinsic value (EUR): 1441.9946532663237


,date,price,action,quantity_mwh,inventory_mwh,cashflow_eur,terminal_liquidation_eur
0,2026-03-01,32.5,1,2.0,52.251256,-65.04,NaN
1,2026-03-02,32.5,1,2.0,54.251256,-65.04,NaN
2,2026-03-03,32.5,1,2.0,56.251256,-65.04,NaN
3,2026-03-04,32.5,1,2.0,58.251256,-65.04,NaN
4,2026-03-05,32.5,1,2.0,60.251256,-65.04,NaN
5,2026-03-06,32.5,1,2.0,62.251256,-65.04,NaN
6,2026-03-07,32.5,1,2.0,64.251256,-65.04,NaN
7,2026-03-08,32.5,1,2.0,66.251256,-65.04,NaN
8,2026-03-09,32.5,1,2.0,68.251256,-65.04,NaN
9,2026-03-10,32.5,1,2.0,70.251256,-65.04,NaN


,date,price,action,quantity_mwh,inventory_mwh,cashflow_eur,terminal_liquidation_eur
355,2027-02-19,30.584,-1,0.0,-0.241206,0.0,NaN
356,2027-02-20,30.584,-1,0.0,-0.241206,0.0,NaN
357,2027-02-21,30.584,-1,0.0,-0.241206,0.0,NaN
358,2027-02-22,30.584,-1,0.0,-0.241206,0.0,NaN
359,2027-02-23,30.584,-1,0.0,-0.241206,0.0,NaN
360,2027-02-24,30.584,-1,0.0,-0.241206,0.0,NaN
361,2027-02-25,30.584,-1,0.0,-0.241206,0.0,NaN
362,2027-02-26,30.584,-1,0.0,-0.241206,0.0,NaN
363,2027-02-27,30.584,-1,0.0,-0.241206,0.0,NaN
364,2027-02-28,30.584,0,0.0,-0.241206,0.0,-7.372221


Number of decision days: 365


intrinsic policy output

In [7]:
out = RESULTS / 'optimal_policy_intrinsic_notebook.csv'
policy.to_csv(out, index=False)
print('Wrote policy to', out)


val_out = RESULTS / 'intrinsic_value_eur.txt'
val_out.write_text(f"{value:.6f}\n")
print('Wrote value to', val_out)


Wrote policy to C:\Users\dappy\Downloads\Public-portfolio\Energy\gas_storage_valuation\results\optimal_policy_intrinsic_notebook.csv
Wrote value to C:\Users\dappy\Downloads\Public-portfolio\Energy\gas_storage_valuation\results\intrinsic_value_eur.txt
